## FASE II

Se presenta a continuación el desarrollo de la actividad correspondiente a la fase II.

Considera que cada producto que se encuentra en el almacén central, además de ser identificado por el tipo de embalaje, también posee como propiedad sus dimensiones (largo, ancho y altura con un máximo de 100,0 cm y un mínimo de 5,0 cm en cada dimensión), peso (kg), el tipo de manipulación que requiere (frágil, normal), lugar de procedencia (A, B, C, D) y temperatura de almacenamiento (ambiente, refrigerado).

Si el vehículo autónomo tiene integrados dos protocolos de manipulación de carga (Protocolo_1 y Protocolo_2), realiza las siguientes operaciones:
1. Genera un conjunto de datos de prueba (mínimo 10,000 muestras), considerando los criterios antes indicados y teniendo en cuenta tu experiencia personal, donde, a partir del análisis de la información del producto, estos puedan ser catalogados dentro los protocolos disponibles.

2. Construye un modelo de aprendizaje automático supervisado a partir del conjunto de datos elaborado, que sea capaz de determinar el tipo de protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.

Importamos las librerías requeridas para ejecutar los requerimientos de los puntos anteriores.* La librería "svd" se encontró en un tutorial de internet relacionado a la generación de datos sintéticos.
[Top 3 Python Packages for Generating Synthetic Data](https://towardsdatascience.com/top-3-python-packages-to-generate-synthetic-data-33a351a5de0c)
* Para esta generación de datos se requiere una pequeña muestra con datos de ejemplo así que se generó un archivo csv llamado "paquetes_fase_II.csv" con una muestra de 10 registros que usa la librería svd para generar los datos sintéticos de 10000.

In [1]:
from sdv.tabular import GaussianCopula #Librería usada para apoyar la creación de datos sintéticos.
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn import metrics
import numpy as np

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/janus78/MasterIA-Notebooks_Tetra3/master/paquetes_fase_II.csv')
data.head()

,Embalaje,Ancho(cm),Largo(cm),Alto(cm),Procedencia,Manipulacion,Temperatura,Protocolo
0,A,35,40,50,A,normal,ambiente,protocolo_1
1,B,80,70,60,B,fragil,refrigerado,protocolo_2
2,C,5,8,4,C,fragil,ambiente,protocolo_2
3,A,25,16,24,B,normal,ambiente,protocolo_1
4,C,12,21,5,A,fragil,refrigerado,protocolo_1


In [3]:
model = GaussianCopula() #Se crea una instancia de la librería "GaussianCopula" para generar datos sintéticos.
model.fit(data) #Se entrena el modelo con los datos de ejemplo.

In [4]:
sample = model.sample(10000) #Se generan datos sintéticos, un total de 10000.
sample.head() #Se muestran los primeros 5 registros.

,Embalaje,Ancho(cm),Largo(cm),Alto(cm),Procedencia,Manipulacion,Temperatura,Protocolo
0,C,23,16,11,C,fragil,refrigerado,protocolo_1
1,C,27,9,5,B,fragil,refrigerado,protocolo_1
2,C,79,68,66,B,normal,refrigerado,protocolo_2
3,B,13,19,21,C,fragil,ambiente,protocolo_2
4,B,66,53,45,A,normal,refrigerado,protocolo_2


In [5]:
# se valida el tamaño de la muestra generada.
print(sample.shape)
print(sample.isna().sum()) #Se verifica que no haya nulos.

(10000, 8)
Embalaje        0
Ancho(cm)       0
Largo(cm)       0
Alto(cm)        0
Procedencia     0
Manipulacion    0
Temperatura     0
Protocolo       0
dtype: int64


In [6]:
from sdv.evaluation import evaluate #Librería usada para evaluar el modelo de generación de datos sintéticos.
evaluate(sample, data, metrics=['CSTest', 'KSTest'], aggregate = False) #Se evalúa el modelo con los datos de ejemplo generados con la librería.

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,0.931448,0.931448,0.0,1.0,MAXIMIZE,None
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.819759,0.819759,0.0,1.0,MAXIMIZE,None


Podemos notar en las métricas mostradas en la celda anterior que la calidad de la generación de datos sintéticos muestra buenos valores en la calidad de los mismos con score entre el 80 y 90%.

* A continuación se va a proceder a categorizar con enteros las columnas que tienen valores string ya que esto facilita la implementación del modelo de aprendizaje automático.

In [7]:
#Se usará el ordinal encoder de sklearn para categorizar las columnas que tienen valores string.
ord_enc = OrdinalEncoder()
sample['Manipulacion_enc'] = ord_enc.fit_transform(sample[['Manipulacion']]) #Se categoriza la columna "Manipulacion" con enteros.
sample['Procedencia_enc'] = ord_enc.fit_transform(sample[['Procedencia']])  #Se categoriza la columna "Procedencia" con enteros.
sample['Embalaje_enc'] = ord_enc.fit_transform(sample[['Embalaje']])    #Se categoriza la columna "Embalaje" con enteros.
sample['Temperatura_enc'] = ord_enc.fit_transform(sample[['Temperatura']])  #Se categoriza la columna "Temperatura" con enteros.
sample['Protocolo_enc'] = ord_enc.fit_transform(sample[['Protocolo']])  #Se categoriza la columna "Protocolo_1" con enteros.
sample.head()

,Embalaje,Ancho(cm),Largo(cm),Alto(cm),Procedencia,Manipulacion,Temperatura,Protocolo,Manipulacion_enc,Procedencia_enc,Embalaje_enc,Temperatura_enc,Protocolo_enc
0,C,23,16,11,C,fragil,refrigerado,protocolo_1,0.0,2.0,2.0,1.0,0.0
1,C,27,9,5,B,fragil,refrigerado,protocolo_1,0.0,1.0,2.0,1.0,0.0
2,C,79,68,66,B,normal,refrigerado,protocolo_2,1.0,1.0,2.0,1.0,1.0
3,B,13,19,21,C,fragil,ambiente,protocolo_2,0.0,2.0,1.0,0.0,1.0
4,B,66,53,45,A,normal,refrigerado,protocolo_2,1.0,0.0,1.0,1.0,1.0


* Se procede a obtener el dataset final "X" con las columnas requeridas para la implementación del modelo de aprendizaje automático.
* Y se obtiene la columna de etiquetas "y" que corresponde al protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.

In [8]:
X = sample[['Manipulacion_enc', 'Procedencia_enc', 'Embalaje_enc', 'Temperatura_enc', 'Ancho(cm)', 'Largo(cm)', 'Alto(cm)']] #Se obtiene el dataset final "X" con las columnas requeridas para la implementación del modelo de aprendizaje automático.
y = sample['Protocolo_enc'] #Se obtiene la columna de etiquetas "y" que corresponde al protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) #Se separan los datos de entrenamiento y prueba.
X.head()

,Manipulacion_enc,Procedencia_enc,Embalaje_enc,Temperatura_enc,Ancho(cm),Largo(cm),Alto(cm)
0,0.0,2.0,2.0,1.0,23,16,11
1,0.0,1.0,2.0,1.0,27,9,5
2,1.0,1.0,2.0,1.0,79,68,66
3,0.0,2.0,1.0,0.0,13,19,21
4,1.0,0.0,1.0,1.0,66,53,45


* Se utiliza el clasificador Perceptron para implementar el modelo de aprendizaje automático.

In [9]:
clf = Perceptron(random_state=0) #Se crea una instancia de la clase Perceptron.
clf.fit(X_train, y_train) #Se entrena el modelo.

Perceptron()

In [10]:
y_pred = clf.predict(X_test) #Se predice el protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.
print(metrics.classification_report(y_test, y_pred)) #Se muestra el reporte de metricas del modelo.

              precision    recall  f1-score   support

         0.0       0.61      0.77      0.68       777
         1.0       0.82      0.68      0.75      1223

    accuracy                           0.72      2000
   macro avg       0.71      0.73      0.71      2000
weighted avg       0.74      0.72      0.72      2000



**Conclusiones:**
Las conclusiones de esta primera parte de la actividad son que podemos darnos cuenta de la importancia de tener una forma con la que generar datos de prueba de calidad y que para encontrarle fue necesario hacer una busqueda que tomó un tiempo considerable.
Además de esto el trabajo usual de realizar el análisis, ajuste y limpieza de datos que tambien toma buena parte del tiempo.
Fue muy ilustrativo realizar esta parte del ejercicioya que nos obligó a buscar nuevas formas de realizar las cosas que se requieren para la actividad.

## Parte II: Pregunta 5 ##
Esta parte corresponde a la pregunta 5 de la actividad. que es relacionada a la distribución de los productos en la planta de manufactura.

1. La planta industrial tiene una distribución como la que se muestra en la figura 1.

   ![Mapeo almacen](https://raw.githubusercontent.com/janus78/MasterIA-Notebooks_Tetra3/master/f2.png)

3. Cada vehículo puede transportar un máximo de tres productos al mismo tiempo.
4. Los productos en el almacén no poseen un orden determinado.
5. Un vehículo puede seleccionar cualquier producto con la misma probabilidad.
6. Elabora un programa que sea capaz de determinar la ruta óptima que debe tomar el vehículo para distribuir los productos por la planta industrial, garantizando que:

7. Nunca repita dos veces el mismo destino en cada viaje.
8. Los productos del protocolo_1 tengan prioridad.
9. La ruta trazada sea siempre la menor.

Para el desarrollo de este ejercicio se ha realizado mucha busqueda, revisión y prácticas de diferentes tutoriales esperando lograr una implementación adecuada. Aún quedan muchas dudas y la inseguridad de que sea correcta la implementación.
A continuación se presenta el desarrollo de este ejercicio.

Dentro de muchos de los tutoriales, guías y consejos que se han revisado se notaba la incidencia de la creación y uso de una matriz de distancias por lo que es la aproximación que se utiliza en este caso.

In [11]:
#Se obtiene una predicción del total de productos en el dataset sintetico.
y_all = clf.predict(X) #Se obtiene la predicción de los protocolos.

In [12]:
X_full = X.copy() #Se crea una copia del dataset original.
X_full['Protocolo_enc'] = y_all #Se agrega la columna de etiquetas al dataset.
X_full.head()

,Manipulacion_enc,Procedencia_enc,Embalaje_enc,Temperatura_enc,Ancho(cm),Largo(cm),Alto(cm),Protocolo_enc
0,0.0,2.0,2.0,1.0,23,16,11,0.0
1,0.0,1.0,2.0,1.0,27,9,5,0.0
2,1.0,1.0,2.0,1.0,79,68,66,0.0
3,0.0,2.0,1.0,0.0,13,19,21,1.0
4,1.0,0.0,1.0,1.0,66,53,45,0.0


In [55]:
#Se obtienen los totales de productos para cada protocolo y embalaje.
total_pt1_A = X_full.apply(lambda x : x['Protocolo_enc'] == 0 and x['Embalaje_enc'] == 0, axis=1).sum() #Se obtiene el total de productos para el protocolo_1 y el embalaje A.
total_pt2_A = X_full.apply(lambda x : x['Protocolo_enc'] == 1 and x['Embalaje_enc'] == 0, axis=1).sum() #Se obtiene el total de productos para el protocolo_2 y el embalaje A.
total_pt1_B = X_full.apply(lambda x : x['Protocolo_enc'] == 0 and x['Embalaje_enc'] == 1, axis=1).sum() #Se obtiene el total de productos para el protocolo_1 y el embalaje B.
total_pt2_B = X_full.apply(lambda x : x['Protocolo_enc'] == 1 and x['Embalaje_enc'] == 1, axis=1).sum() #Se obtiene el total de productos para el protocolo_2 y el embalaje B.
total_pt1_C = X_full.apply(lambda x : x['Protocolo_enc'] == 0 and x['Embalaje_enc'] == 2, axis=1).sum() #Se obtiene el total de productos para el protocolo_1 y el embalaje C.
total_pt2_C = X_full.apply(lambda x : x['Protocolo_enc'] == 1 and x['Embalaje_enc'] == 2, axis=1).sum() #Se obtiene el total de productos para el protocolo_2 y el embalaje C.

print(f"Los totales por embalaje y protocolo son:\nEmbalaje A, Protocolo 1: {total_pt1_A}\nEmbalaje A, Protocolo 2: {total_pt2_A}\nEmbalaje B, Protocolo 1: {total_pt1_B}\nEmbalaje B, Protocolo 2: {total_pt2_B}\nEmbalaje C, Protocolo 1: {total_pt1_C}\nEmbalaje C, Protocolo 2: {total_pt2_C}") #Se muestra el total de productos para cada protocolo y embalaje.

Los totales por embalaje y protocolo son:
Embalaje A, Protocolo 1: 2257
Embalaje A, Protocolo 2: 1526
Embalaje B, Protocolo 1: 2085
Embalaje B, Protocolo 2: 2291
Embalaje C, Protocolo 1: 710
Embalaje C, Protocolo 2: 1131


In [56]:
#Creando arreglo con el numero de paquetes por punto de entrega
demandas = pd.DataFrame([[total_pt1_A, total_pt2_A, total_pt1_B, total_pt2_B, total_pt1_C, total_pt2_C]], columns=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']) #Se crea un dataframe con el numero de paquetes por punto de entrega.
demandas

,A1,A2,B1,B2,C1,C2
0,2257,1526,2085,2291,710,1131


In [57]:
#Importando las librerias necesarias.
from scipy.spatial import distance_matrix #Se importa la libreria de distancia.
from ortools.constraint_solver import routing_enums_pb2 #Se importa la libreria de routing de ortools Google.
from ortools.constraint_solver import pywrapcp #Se importa la libreria de routing de ortools Google.
from python_tsp.exact import solve_tsp_dynamic_programming #Otra alternativa para encontrar la ruta mas corta y su peso

In [58]:
#Creando el método para la generación de la matriz de distancias.
def distancia_matriz(arreglo_coordenadas):
    dist_matrix = distance_matrix(arreglo_coordenadas.values, arreglo_coordenadas.values).astype(int) #Se calcula la matriz de distancias.
    return dist_matrix #Se retorna la matriz de distancias.

In [59]:
#Definiendo puntos de entrega en el almacen
A1 = [100, 600] #Punto de entrega embalaje A protocolo 1
A2 = [400, 600] #Punto de entrega embalaje A protocolo 2
B1 = [700, 0] #Punto de entrega embalaje B protocolo 1
B2 = [400, 0] #Punto de entrega embalaje B protocolo 2
C1 = [1000, 200] #Punto de entrega embalaje C protocolo 1
C2 = [1000, 300] #Punto de entrega embalaje C protocolo 2

A continuación se definen metodos y pasos para obtener la ruta más corta y el total recorrido usando Google OR Tools

In [60]:
#Creando el modelo de datos
def create_data_model(distance_matrix):
    data = {'distance_matrix': distance_matrix, 'num_vehicles': 1, 'depot': 0}  #Se crea un diccionario vacio.
    return data #Se retorna el diccionario.

In [61]:
#Creando método para imprimir y regresar el valor de la ruta más corta.
def print_solution(manager, routing, solution):
    print('Objective: {} m'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Ruta para el vehiculo 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Distancia de la ruta: {}m\n'.format(route_distance)
    return route_distance #Se retorna la distancia de la ruta.

In [62]:
#Proceso ruta mas corta
def get_route(distance_matrix_initial):
    data = create_data_model(distance_matrix_initial) #Se crea el modelo de datos.

    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot']) #Se crea el manejador de rutas.

    routing = pywrapcp.RoutingModel(manager) #Se crea el modelo de rutas.

    routing.SetVehicleUsedWhenEmpty(True, 0) #Se considera el costo de la ruta vacía.

    #Creando el metodo de distancias
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback) #Se registra el callback de distancias.

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index) #Se establece el callback de distancias.

    #Obteniendo la primera solución heurística
    search_parameters = pywrapcp.DefaultRoutingSearchParameters() #Se crea el objeto de búsqueda.
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 10 #Se establece el tiempo limite de búsqueda.
    search_parameters.log_search = True #Se establece el log de búsqueda.

    #Solución
    solution = routing.SolveWithParameters(search_parameters) #Se obtiene la solución.

    #Imprimiendo la solución y obteniendo el valor
    if solution:
        return print_solution(manager, routing, solution) #Se imprime la solución.
    else:
        return 0 #Se retorna nulo.
        print('No se pudo obtener una solución') #Se imprime que no se pudo obtener una solución.

A continuación se definen metodos y pasos para obtener la ruta más corta y el total recorrido para todos los paquetes de los diferentes embalajes y protocolos. Se tiene un arreglo de prioridades para cada embalaje y protocolo. Se le da preferencia al protocolo 1.

In [63]:
not_completed = True #Se crea una variable para saber si todos los paquetes fueron entregados.
ruta_total = 0 #Se crea una variable para saber la distancia total recorrida.
probability_array = [0.3, 0.034, 0.3, 0.033, 0.3, 0.033] #Se crea un arreglo para guardar las probabilidades de seleccionar protocolo 1 y darle mayor prioridad, la suma de las probabilidades debe ser 1.
while not_completed:
    if len(demandas.columns) <= 0: #Se valida que todos los paquetes fueron entregados.
        not_completed = False
        continue
    demanda_array = demandas.columns.values #Se obtiene el arreglo de las demandas.
    packages = np.random.choice(demanda_array, 3, replace=True, p=probability_array) #Se obtienen 3 paquetes aleatorios dando prioridad a los del protocolo 1.
    paquetes = packages.tolist() #Se obtiene los nombres de las columnas.
    dict_columns = {item: paquetes.count(item) for item in paquetes} #Se crea un diccionario con los nombres de las columnas y el número de paquetes.

    #print(packages, paquetes, dict_columns) #Se imprimen los datos.

    coords_demanda = [[0, 300]] #Se crea una lista para guardar las coordenadas de los paquetes. El primer elemento es la ubicación del depósito.

    for key in dict_columns:
        if key == 'A1':
            coords_demanda.append(A1)
            total_pt1_A -= dict_columns[key] #Se resta el número de paquetes de la columna A1.
            if total_pt1_A <= 0: #Se balancearan las probabilidades antes de eliminar la columna completada de reparto
                A1_prob = probability_array[demandas.columns.get_loc('A1')] #Se obtiene la probabilidad de la columna A1.
                if 'B1' in demandas.columns:
                    probability_array[demandas.columns.get_loc('B1')] = A1_prob + probability_array[demandas.columns.get_loc('B1')] #Se le suma la probabilidad de la columna B1.
                    probability_array.pop(demandas.columns.get_loc('A1')) #Se elimina la probabilidad de seleccionar la columna A1.
                elif 'C1' in demandas.columns:
                    probability_array[demandas.columns.get_loc('C1')] = A1_prob + probability_array[demandas.columns.get_loc('C1')] #Se le suma la probabilidad de la columna C1.
                    probability_array.pop(demandas.columns.get_loc('A1')) #Se elimina la probabilidad de seleccionar la columna A1.
                else:
                    probability_array.pop(demandas.columns.get_loc('A1')) #Se elimina la probabilidad de seleccionar la columna A1.
                    if probability_array:
                        probability_array[0] = A1_prob + probability_array[0] #Se le suma la probabilidad de la columna A1.
                demandas.drop('A1', axis=1, inplace=True) #Se elimina la columna A1 del reparto.
        elif key == 'A2':
            coords_demanda.append(A2)
            total_pt2_A -= dict_columns[key] #Se resta el número de paquetes de la columna A2.
            if total_pt2_A <= 0:  #Se balancearan las probabilidades antes de eliminar la columna completada de reparto
                A2_prob = probability_array[demandas.columns.get_loc('A2')] #Se obtiene la probabilidad de la columna A2.
                probability_array.pop(demandas.columns.get_loc('A2')) #Se elimina la probabilidad de seleccionar la columna A2.
                if probability_array:
                    probability_array[0] = A2_prob + probability_array[0] #Se le suma la probabilidad de la columna A2.
                demandas.drop('A2', axis=1, inplace=True) #Se elimina la columna A2 del reparto.
        elif key == 'B1':
            coords_demanda.append(B1)
            total_pt1_B -= dict_columns[key] #Se resta el número de paquetes de la columna B1.
            if total_pt1_B <= 0: #Se balancearan las probabilidades antes de eliminar la columna completada de reparto
                B1_prob = probability_array[demandas.columns.get_loc('B1')] #Se obtiene la probabilidad de la columna B1.
                if 'A1' in demandas.columns:
                    probability_array[demandas.columns.get_loc('A1')] = B1_prob + probability_array[demandas.columns.get_loc('A1')] #Se le suma la probabilidad de la columna A1.
                    probability_array.pop(demandas.columns.get_loc('B1')) #Se elimina la probabilidad de seleccionar la columna B1.
                elif 'C1' in demandas.columns:
                    probability_array[demandas.columns.get_loc('C1')] = B1_prob + probability_array[demandas.columns.get_loc('C1')] #Se le suma la probabilidad de la columna C1.
                    probability_array.pop(demandas.columns.get_loc('B1')) #Se elimina la probabilidad de seleccionar la columna B1.
                else:
                    probability_array.pop(demandas.columns.get_loc('B1')) #Se elimina la probabilidad de seleccionar la columna B1.
                    if probability_array:
                        probability_array[0] = B1_prob + probability_array[0] #Se le suma la probabilidad de la columna B1.
                demandas.drop('B1', axis=1, inplace=True) #Se elimina la columna B1 del reparto.
        elif key == 'B2':
            coords_demanda.append(B2)
            total_pt2_B -= dict_columns[key] #Se resta el número de paquetes de la columna B2.
            if total_pt2_B <= 0: #Se balancearan las probabilidades antes de eliminar la columna completada de reparto
                B2_prob = probability_array[demandas.columns.get_loc('B2')] #Se obtiene la probabilidad de la columna B2.
                probability_array.pop(demandas.columns.get_loc('B2')) #Se elimina la probabilidad de seleccionar la columna B2.
                if probability_array:
                    probability_array[0] = B2_prob + probability_array[0] #Se le suma la probabilidad de la columna B2.
                demandas.drop('B2', axis=1, inplace=True) #Se elimina la columna B2 del reparto.
        elif key == 'C1':
            coords_demanda.append(C1)
            total_pt1_C -= dict_columns[key] #Se resta el número de paquetes de la columna C1.
            if total_pt1_C <= 0: #Se balancearan las probabilidades antes de eliminar la columna completada de reparto
                C1_prob = probability_array[demandas.columns.get_loc('C1')] #Se obtiene la probabilidad de la columna C1.
                if 'A1' in demandas.columns:
                    probability_array[demandas.columns.get_loc('A1')] = C1_prob + probability_array[demandas.columns.get_loc('A1')] #Se le suma la probabilidad de la columna C1.
                    probability_array.pop(demandas.columns.get_loc('C1')) #Se elimina la probabilidad de seleccionar la columna B1.
                elif 'B1' in demandas.columns:
                    probability_array[demandas.columns.get_loc('B1')] = B1_prob + probability_array[demandas.columns.get_loc('C1')] #Se le suma la probabilidad de la columna B1.
                    probability_array.pop(demandas.columns.get_loc('C1')) #Se elimina la probabilidad de seleccionar la columna C1.
                else:
                    probability_array.pop(demandas.columns.get_loc('C1')) #Se elimina la probabilidad de seleccionar la columna C1.
                    if probability_array:
                        probability_array[0] = C1_prob + probability_array[0] #Se le suma la probabilidad de la columna C1.
                demandas.drop('C1', axis=1, inplace=True) #Se elimina la columna C1 del reparto.
        elif key == 'C2':
            coords_demanda.append(C2)
            total_pt2_C -= dict_columns[key] #Se resta el número de paquetes de la columna C2.
            if total_pt2_C <= 0: #Se balancearan las probabilidades antes de eliminar la columna completada de reparto
                C2_prob = probability_array[demandas.columns.get_loc('C2')] #Se obtiene la probabilidad de la columna B2.
                probability_array.pop(demandas.columns.get_loc('C2')) #Se elimina la probabilidad de seleccionar la columna B2.
                if probability_array:
                    probability_array[0] = B2_prob + probability_array[0] #Se le suma la probabilidad de la columna B2.
                demandas.drop('C2', axis=1, inplace=True) #Se elimina la columna C2 del reparto.

    #print(coords_demanda) #Se imprime la lista de coordenadas.

    df_coords_demanda = pd.DataFrame(coords_demanda, columns=['x', 'y']) #Se crea un dataframe con las coordenadas de los paquetes.
    #print(df_coords_demanda) #Se imprime el dataframe.

    demanda_distance_matrix = distancia_matriz(df_coords_demanda) #Se obtiene la matriz de distancias.

    #print(demanda_distance_matrix)#Se obtiene la matriz de distancias.

    #ruta_total += get_route(df_coords_demanda) Este usa la implementación de la librería de google y esa regresa la ruta pero no el costo. Se deja como referencia por si se quiere probar esta forma
    permutation,  distance = solve_tsp_dynamic_programming(demanda_distance_matrix) #Se obtiene la ruta y el costo de la ruta. Se usa la implementación de la librería python_tsp
    ruta_total += distance #Se suma el costo de la ruta.
    #print('Rute', permutation + [0]) #Se imprime la ruta.
    #print('Distancia de ruta', distance) #Se imprime el costo de la ruta.
    #print(total_pt1_A, total_pt2_A, total_pt1_B, total_pt2_B, total_pt1_C, total_pt2_C) #Se imprime los totales de paquetes por cada punto.
ruta_total = "{:,}".format(ruta_total) #Se formatea el costo de la ruta.
print(f"La ruta total recorrida por el vehiculo para entregar todos los productos con prioridad de protocolo 1 es:\n{ruta_total}m") #Se imprime la distancia total recorrida.

La ruta total recorrida por el vehiculo para entregar todos los productos con prioridad de protocolo 1 es:
5,335,993m
